In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import math
import time
import csv


In [2]:
df_aed = pd.read_parquet('aed_locations.parquet.gzip')

In [3]:
len(df_aed)

15227

In [4]:
#create address as one string 
number = 1
addresses = []
for number in range(len(df_aed)):
    address = ''
    if not pd.isna(df_aed['address'][number]):
        address = str(df_aed['address'][number])
        if not pd.isna(df_aed['number'][number]):
            address = address+" "+str(int(df_aed['number'][number]))
        if not pd.isna(df_aed['postal_code'][number]):
            address = address +" ,"+str(int(df_aed['postal_code'][number]))
        if not pd.isna(df_aed['municipality'][number]):
            address = address +" ,"+str(df_aed['municipality'][number])
        address = address + ' , Belgium'
        addresses.append(address)

In [77]:
def getting_coordinate(starting_value):
    counter = starting_value
    coordinates = []
    for i in range(3):
        some_locations = gpd.tools.geocode(addresses[counter:counter+20])
        some_coordinates = some_locations.get_coordinates()
        print(i)
        for index,row in some_coordinates.iterrows():
            coordinates.append(row)
        counter += 20
        time.sleep(10)
    return coordinates


In [115]:
counter = 1215+100+99+100+57+100+100+100+40+57+59+180+158+178+298+197+199+199+199+59
coordinates = []
for i in range(10):
    some_locations = gpd.tools.geocode(addresses[counter:counter+20])
    some_coordinates = some_locations.get_coordinates()
    print(i)
    for index,row in some_coordinates.iterrows():
        coordinates.append(row)
    counter += 21
    time.sleep(5)

0
1
2


GeocoderUnavailable: HTTPSConnectionPool(host='photon.komoot.io', port=443): Max retries exceeded with url: /api?q=Rue+Barbet++86+%2C7301+%2CBoussu+%2C+Belgium&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='photon.komoot.io', port=443): Read timed out. (read timeout=1)"))

In [123]:
len(coordinates)
#we have 1215 coordinates right now
#aed_coordinates 1 2 3 
count = 100+99+100+57+100+100+100+40+57+59+180+79+158+178+1215 +298+197 +199+199+199+59
count

3773

In [99]:
#c4 =[]

In [117]:
c4 += coordinates

In [118]:
len(c4)

853

In [ ]:
#c3 = []

In [ ]:
for i in range(2622+298,5000,60):
    coords = getting_coordinate(i)
    c3 += coords
    print(len(coords))
    print(len(c3))
    time.sleep(3*60)
    print('done waiting')

In [83]:
len(c3)

298

In [ ]:
#aed_coordinates1 = getting_coordinate(1)
#aed_coordinates2 = getting_coordinate(120)
#aed_coordinates3 = getting_coordinate(220)
#aed_coordinates4 = getting_coordinate(320)
#aed_coordinates5 = getting_coordinate(420)
#aed_coordinates6 = getting_coordinate(520)
#aed_coordinates7 = getting_coordinate(620)
#aed_coordinates8 = getting_coordinate(720)
#aed_coordinates9 = getting_coordinate(820)
#aed_coordinates10 = getting_coordinate(920)
#aed_coordinates11 = getting_coordinate(1020)
#aed_coordinates12 = getting_coordinate(1120)
#aed_coordinates13 = getting_coordinate(1220)
#aed_coordinates14 = getting_coordinate(1320)
#aed_coordinates15 = getting_coordinate(1420)

In [119]:
#change it to 'aed_coordinates2.csv' etc.
file_path = 'aed_coordinates4.csv'

In [120]:
# Write the list to a CSV file
with open(file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerows(c4)

print("CSV file saved successfully.")

CSV file saved successfully.


In [121]:
column_names = ['lon', 'lat']

# Read the CSV file into a pandas DataFrame
aed_coordinates4 = pd.read_csv(file_path, names = column_names)


merge aed files and look for duplicates


In [122]:
# Combine the two DataFrames
all_coordinates_uncleaned= pd.concat([aed_coordinates1, aed_coordinates2, aed_coordinates3, aed_coordinates4])
print(len(all_coordinates_uncleaned))
# Drop exact duplicate rows
all_coordinates = all_coordinates_uncleaned.drop_duplicates(keep='first')

# Display the combined DataFrame
len(all_coordinates)

3773


3132

Make the map with the coordinates of the AEDs

In [124]:
px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 7       # Zoom level (1-20)
# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
aed_location_layer = px.scatter_mapbox(all_coordinates, lat='lat', lon='lon')

# Add the scatter plot layer to the base map
map_with_aed_points = map_base.add_trace(aed_location_layer.data[0])

# Show the map
map_with_aed_points.show()